In [210]:
import math
from math import log2
import pandas as pd
from statistics import variance, mean, mode
import numpy as np
import random

# Distances

In [196]:
#Manhattan
def manhattan_dist(p1: tuple,p2: tuple):
    val = 0
    for i in range(len(p1)):
        val += abs(p1[i]-p2[i])
    return val

In [197]:
#Euclidean
def euclidean_dist(p1: tuple,p2: tuple):
    val = 0
    for i in range(len(p1)):
        val += (p1[i]-p2[i])**2
    return math.sqrt(val)

In [198]:
#Chebyshev
def chebyshev_dist(p1: tuple,p2: tuple):
    return max([abs(p1[i]-p2[i]) for i in range(len(p1))])

# Normalization

In [199]:
#Min-Max-Norm
def min_max_norm(l: list):
    ma = max(l)
    mi = min(l)
    li = [(i - mi)/(ma-mi) for i in l]
    return li

In [200]:
#Z-Norm
def z_norm(l: list):
    v = variance(l)
    m = mean(l)
    li = [(i-m)/math.sqrt(v) for i in l]
    return li

# Metrics

In [201]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

#Call these functions with the arguments (ground-truth, prediction)

# Entropy, Information Gain, Gain Ratio, Gini Index

## Entropy and information gain

In [202]:
#Entropy
def entropy(entropy_row, class_row):
    data = pd.concat([entropy_row, class_row], axis=1)
    data.set_axis(['Entropy', 'Class'], axis=1, inplace=True)
    data = data.groupby([l for l in data.columns]).size().reset_index(name='Count')

    dic = {}
    for val in entropy_row.unique():
        d = data[data.Entropy == val]
        l = sum(d["Count"])
        dic[val] = -sum([((i/l)*log2(i/l)) for i in d["Count"]])

    e = 0  
    l = len(class_row)  
    for i in class_row.value_counts():
        e += (-(i/l)*log2(i/l))
    return dic, e

#Information Gain
def inf_gain(entropy_row, class_row):
    ent, whole_set = entropy(entropy_row,class_row)
    l = len(entropy_row)
    t = entropy_row.value_counts()
    
    for val in entropy_row.unique():
        whole_set -= ((t[val]/l)*ent[val])

    return whole_set

d = {'Genre': ["Action", "Action", "Action", "Adventure","Adventure", "Adventure","Adventure", "Adventure","Adventure", "Adventure"],
    'Buy': ["No","No","No","Yes","Yes","No","No","Yes","Yes","No"]}
df = pd.DataFrame(data=d)
 
#inf_gain(df.Genre, df.Buy)

## Gain ratio

## Gini index

# Classification

## k-NN

In [206]:
def distance_matrix(data, dist_type):
    data_ex_class = [p[:-1] for p in data]
    mat = [[] for i in range(len(data))]

    for i in range(len(data)):
        for j in range((i),len(data)):
            if i == j:
                #Not "Correct" with inf, but we want to find minimum in the next step, excluding distance to self
                mat[i].append(math.inf)
            else:
                d = dist_type(data_ex_class[i],data_ex_class[j])
                mat[i].append(d)
                mat[j].append(d)
    
    return mat

def find_k_min(d_matr, k):
    mat = d_matr.copy()
    l = []
    for i in range(len(mat)):
        x = np.array(mat[i]).argsort()[:k]
        l.append(x)
    return l

def k_nn(data, k, dist_type): 
    d_matr = distance_matrix(data, dist_type)
    indexes = find_k_min(d_matr, k)
    class_at_ind = [[data[i][-1] for i in j] for j in indexes]
    pred = [max(set(class_at_ind[i]), key=class_at_ind[i].count) for i in range(len(class_at_ind))]
    return pred
    

#(coord, ..., coord, class)
#data = [(1,0.4,0), (1.3,-0.4,0),(1.1,0.8,0),(1,-2,0),(-0.1,0,0),(0,0.3,1),(-0.6,0.9,1),(-1.4,-1.4,1),(-1.3,-0.2,1),(-1,1.5,1)]
#k_nn(data, 3, manhattan_dist)

## Naïve Bayes

# k-Means and SSE

In [286]:
def dist_cent_point(data, centroids, dist_type):
    l = [[] for i in range(len(data))]
    for i in range(len(centroids)):
        for j in range(len(data)):
            l[j].append(dist_type(data[j],centroids[i]))
    return l

#Returns the new centroid with the points associated
def compute_centroids_points(data, dist,median):
    d = [[] for i in range(len(dist[0]))]
    for i in range(len(data)):

        ### THIS SHOULD BE AMENDED TO PURSUE EACH PATH AND FIND THE ONE WITH LOWEST SSE ###
        ### Dilemma: Spend time fixing, or just run ~ 100 times and pick solution with lowest SSE? ###
        ind = random.choice(np.where(np.array(dist[i]) == np.array(dist[i]).min())[0])

        d[ind].append(data[i])
    dic = {}
    for points in d:
        if median:
            x = np.median(points, axis=0)
        else:
            x = np.average(points, axis=0)
        dic[tuple(x)] = points 
    return dic

def sse(centroids_with_points: dict, dist_type):
    s = 0 
    for key,vals in centroids_with_points.items():
        for val in vals:
            s += dist_type(key,val)**2
    return s
    
#returns result in i iterations
def k_means(data, centroids_or_k, dist_type, median):
    if type(centroids_or_k) == int:
        centroids = random.sample(data,centroids_or_k)
    else: 
        centroids = centroids_or_k
    i = 0
    while True:
        dist = dist_cent_point(data, centroids, dist_type)
        new_c_p = (compute_centroids_points(data,dist,median))
        i += 1
        new_centroids = [i for i in new_c_p.keys()]
        if set(new_centroids) == set(centroids):
            return new_c_p, i
        centroids = [i for i in new_c_p.keys()]
        
#Because of randomness in picking between ties --> run n times 
def k_means_runner(data, centroids_or_k, dist_type, n, median):
    sol = (None,math.inf,0) #(Solution, SSE, Iterations)
    #Iterations to run given by num
    for i in range(n):
        r,i = k_means(data,centroids_or_k,manhattan_dist, median=True)
        s = sse(r,manhattan_dist)
        if s < sol[1]:
            sol = (r,s,i) 
    return sol


# median decides how to compute new centroid --> TRUE: Median, FALSE: AV
data = [(2,2),(4,6),(4,8),(6,6),(6,8),(8,0)]
centroids = [(4,6),(4,8),(6,6)]
k_means_runner(data,centroids,manhattan_dist, 50, median=True)

({(2.0, 2.0): [(2, 2)],
  (5.0, 7.0): [(4, 6), (4, 8), (6, 6), (6, 8)],
  (8.0, 0.0): [(8, 0)]},
 16.0,
 3)

# RUN BLOCK (RUN ALL FIRST)